# Combined AlphaFold 3 + Boltz-2 Analysis

This notebook demonstrates the full pipeline combining both models.

In [ ]:
import sys
sys.path.append('..')

from src.pipeline import ProteinPipeline
from src.visualization import StructureViewer, ConfidencePlotter

## 1. Initialize Pipeline

In [ ]:
pipeline = ProteinPipeline(
    alphafold3_weights='../models/alphafold3',
    boltz2_weights='../models/boltz2'
)

## 2. Run Complete Analysis

In [ ]:
# Input data
sequence = 'MKFLKFSLLTAVLLSVVFAFSSCGDDDDTGYLPPSQAIQDLLKRMKV'
ligands = [
    'CC(=O)OC1=CC=CC=C1C(=O)O',  # Aspirin
    'CC(C)CC1=CC=C(C=C1)C(C)C(=O)O'  # Ibuprofen
]

# Run pipeline
results = pipeline.run(
    sequence=sequence,
    ligands=ligands,
    generate_report=True,
    experiment_name='demo_analysis'
)

print(f'Pipeline completed in {results.execution_time:.2f}s')
print(f'Report: {results.report_path}')

## 3. Compare Results

In [ ]:
# AlphaFold 3 results
if results.af3_result:
    print('AlphaFold 3:')
    print(f'  Mean pLDDT: {results.af3_result.plddt.mean():.2f}')
    print(f'  pTM: {results.af3_result.ptm:.3f}')

# Boltz-2 results
print('\nBoltz-2 Affinity Predictions:')
for i, result in enumerate(results.boltz2_results, 1):
    print(f'  Ligand {i}:')
    print(f'    IC50: {result.ic50:.2f} nM')
    print(f'    ΔG: {result.delta_g:.2f} kcal/mol')

## 4. Visualize Best Binding Pose

In [ ]:
# Find best affinity
best_idx = min(range(len(results.boltz2_results)), 
               key=lambda i: results.boltz2_results[i].ic50)

best_result = results.boltz2_results[best_idx]

print(f'Best ligand: #{best_idx + 1}')
print(f'IC50: {best_result.ic50:.2f} nM')

# Visualize
viewer = StructureViewer()
view = viewer.show_complex(best_result.pdb_path)
view.show()